## Introduction

Google EarthEngine *GEE* is a remote sensing repository with steroids. So it is much more than just a storage service. 

GEE allows you to:

1) Query, retrieve, and clip specific remote sensing and vector data for your needs (area of study). As any repository should do.

2) Process/filter the remote sensing data before bringing it to your local or HydroShare storage. For example, clouded pixels can be "filtered" and NDVI values calculated before importing the e.g. timeseries files.

3) Connect the EarthEngine datasets to live applications (e.g. an app). An example of this is the EEFlux project: https://eeflux-level1.appspot.com/

We need to learn some of the GEE terminology to get it to work. This notebook will be your guide. Let's begin:

## Visualizing Images and Images Bands:

Visualizing Images using GEE requires the use of the ipyleaflet module. 


In [35]:
# Initializing display, earthengine, and ipyleaflet
from IPython.display import Image
%matplotlib inline

import ee
ee.Initialize()

import ipyleaflet
from ipyleaflet import *

m = ipyleaflet.Map(center=(41.7370,-111.8338),zoom=0)
m.add_control(ipyleaflet.LayersControl())

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

### Visualizing an image

In [36]:
# Instantiate an image with the Image constructor.
image = ee.Image('CGIAR/SRTM90_V4');
# get image GEE Id
map_id = image.getMapId()

image_tile_url = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}".format(**map_id)
image_lyr = ipyleaflet.TileLayer(url=image_tile_url,name="CGIAR/SRTM90_V4")

# Display the image on the map above.
m.add_layer(image_lyr);

Observe that a very gray image appears on the map. Don't worry, you'll make it look better soon.

## Digression: Images in Earth Engine
Images in Earth Engine (see this page for more details) are made up of one or more bands. Each band in an image has its own name, pixel values, pixel resolution, and projection. As you'll soon discover, the SRTM image has one band: 'elevation'.

When you add an image to a map using *m.add_layer()*, Earth Engine needs to determine how to map the values in the image band(s) to colors on the display. If a single-band image is added to a map, by default Earth Engine displays the band in grayscale, where the minimum value is assigned to black, and the maximum value is assigned to white. If you don't specify what the minimum and maximum should be, Earth Engine will use default values. For example, the image you just added to the map is displayed as a grayscale image stretched to the full range of the data, or signed 16-bit integer [-32768, 32767]. (float bands are stretched to [0, 1] and byte bands are stretched to [0, 255] by default).

You can discover the data type of the image by printing it and inspecting the image object in the Console tab. For example, run the following after the previous code:

In [37]:
print('SRTM image', image.bandNames().getInfo());
print('SRTM image', image.bandTypes().getInfo());

SRTM image ['elevation']
SRTM image {'elevation': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}}


See that there is only one bands called 'elevation' band at index '0' and the 'data_type' property of the 'elevation' band is a signed int16 data type.

## Customizing layer visualization
To change the way the data are stretched, you can provide another parameter to the Map.addLayer() call. Specifically, the second parameter, visParams, lets you specify the minimum and maximum values to display. To discover what values to use, activate the Inspector tab and click around on the map to get an idea of the range of pixel values. Alternatively, use the Layer manager to interactively stretch the data, then observe the minimum and maximum corresponding to percentiles or standard deviation stretches. Suppose that through such experimentation, you determine that the data should be stretched to [0, 3000]. To display the image using this range, use:

In [38]:
image2=image.visualize(min=0,max=3000)

# get image GEE Id
map_id = image2.getMapId()

image_tile_url = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}".format(**map_id)
image_lyr = ipyleaflet.TileLayer(url=image_tile_url,name="CGIAR/SRTM90_V4 stretched")

# Display the image on the map above.
m.add_layer(image_lyr);

To display a single band using a color palette, add a palette property to the visParams object. We can use the seaborn python module to create hexadecimal palettes. To know more about paletes, see this link: https://chrisalbon.com/python/data_visualization/seaborn_color_palettes/


In [39]:
import seaborn as sns

In [43]:
# color ramp based on hexadecimal values:
pal = sns.color_palette('cubehelix', 5)
palette =pal.as_hex()

image3=image.visualize(min=0,max=3000,palette= palette)

# get image GEE Id
map_id = image3.getMapId()

image_tile_url = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}".format(**map_id)
image_lyr = ipyleaflet.TileLayer(url=image_tile_url,name="CGIAR/SRTM90_V4 colormap")

# Display the image on the map above.
m.add_layer(image_lyr);